In [2]:
import pandas as pd
import torch


feature_df = pd.read_csv("./data/27_features_rets_normalized.csv")
feature_df


,permno,time_avail_m,IdioVolAHT,IdioRisk,IdioVol3F,IdioVolCAPM,MaxRet,ReturnSkew,ReturnSkew3F,hire,...,Mom6m,NumEarnIncrease,DivOmit,DivInit,IndIPO,realestate,ConvDebt,secured,securedind,RET
0,10006,196501,0.012778,-0.601212,-0.571781,-0.626732,-0.547343,0.245102,0.120902,-0.413000,...,-0.008222,3.054562,-0.240517,-0.263257,NaN,NaN,-0.278337,NaN,NaN,-0.183063
1,10014,196501,0.034409,0.551295,0.532243,0.512546,0.007272,-0.428604,-0.125614,-0.177911,...,-0.614532,-0.327213,-0.240517,-0.263257,NaN,NaN,-0.278337,NaN,NaN,-0.216503
2,10030,196501,0.008101,-0.897249,-0.956225,-0.931466,-0.795607,-1.703301,-0.153717,-0.177911,...,-0.071137,-0.327213,-0.240517,-0.263257,NaN,NaN,-0.278337,NaN,NaN,-0.231768
3,10057,196501,0.011245,-0.379386,-0.346098,-0.374121,-0.479644,-0.950473,-0.599847,-0.177911,...,-0.144080,-0.327213,-0.240517,-0.263257,NaN,NaN,-0.278337,NaN,NaN,-0.314581
4,10102,196501,0.009294,-0.740124,-0.754605,-0.757764,-0.695667,-0.486146,-0.487812,0.136847,...,0.038258,3.730917,-0.240517,-0.263257,NaN,NaN,3.590943,NaN,NaN,0.174906
5,10137,196501,0.011147,-0.631659,-0.586596,-0.625962,-0.697153,-1.078332,-1.369574,-0.234499,...,0.082704,-0.327213,-0.240517,-0.263257,NaN,NaN,-0.278337,NaN,NaN,-0.314890
6,10145,196501,0.009373,-0.761163,-0.787222,-0.788559,-0.636865,-0.812758,-0.970558,0.251957,...,-0.190643,-0.327213,-0.240517,-0.263257,NaN,NaN,-0.278337,NaN,NaN,0.306665
7,10153,196501,0.015791,-0.055916,-0.045437,-0.057695,0.195971,0.993685,1.100076,0.020883,...,-0.318526,3.054562,-0.240517,-0.263257,NaN,NaN,-0.278337,NaN,NaN,0.759367
8,10161,196501,0.014037,-0.280630,-0.358212,-0.330422,-0.246596,-0.421855,0.825192,0.887426,...,-0.300819,4.407272,-0.240517,-0.263257,NaN,NaN,-0.278337,NaN,NaN,-0.164357
9,10188,196501,0.017665,-0.129262,-0.239518,-0.154097,-0.268842,-0.045869,-1.221741,-2.614750,...,1.162455,-0.327213,-0.240517,-0.263257,NaN,NaN,-0.278337,NaN,NaN,-0.167025


In [3]:
test = feature_df.fillna(method="backfill",axis=0)

In [5]:
test.to_csv("./data/27_features_rets_normalized_filled.csv",index=False)

In [21]:
def get_feature_data_by_date(feature_df, date):
    next_month = (date + 1) if (date % 100 < 12) else (date + 89)
    ret = feature_df[feature_df["time_avail_m"] == next_month][["permno","RET"]]
    ret_cp = ret.copy()
    feature = feature_df[feature_df["time_avail_m"] == date][feature_df.columns[0:29]]
    feature_cp = feature.copy()
    merge_df = pd.merge(feature_cp, ret_cp, left_on=["permno"],right_on=["permno"])

    merge_df.drop(merge_df.columns[[0, 1]], axis=1, inplace=True)
    
    feature_list = []
    ret_list = []
    for index, row in merge_df.iterrows():
        array = row.values
        feature = array[0:27]
        ret = array[27]
        feature_list.append(feature)
        
        ret_list.append(ret)
        
    feature_array = np.array(feature_list)
    ret_array = np.array(ret_list)
    return feature_array,ret_array

In [22]:
def get_macro_by_date(macro_df, begin_date, end_date):
    if (begin_date == end_date):
        sub_df = macro_df[macro_df["sasdate"]==begin_date]
        sub_df = sub_df.drop("sasdate", axis=1)
        out = sub_df.values
        return out
    else:
        sub_df = macro_df[(macro_df["sasdate"] >= begin_date)
                          & (macro_df["sasdate"] <= end_date)]

        sub_df = sub_df.drop("sasdate", axis=1)
        out = sub_df.values
        return out

In [5]:
import sys
import torch
from Network.lstm import LSTM
from Network.conditional_net import ConditionalNet
from Network.feedforward_net import FeedforwardNet
from torch import nn
import pandas as pd


In [9]:
MACRO_SIZE = 124
LSTM_HIDDEN = 128
LSTM_LAYER = 1
CHAR_SIZE = 27
FF_HIDDEN = [32, 64, 32]
FF_LAYER = 3
BATCH_SIZE = 3

In [39]:
import torch
import torch.nn as nn
from Network.lstm import LSTM
from Network.feedforward_net import FeedforwardNet

class ConditionalNet(nn.Module):
    def __init__(self, macro_size, char_size, 
                 lstm_hidden_size, lstm_num_layers, 
                 ff_hidden_size, ff_num_layers):
        super().__init__()
        self.MomentRNN = LSTM(macro_size, lstm_hidden_size, lstm_num_layers)
        ff_input_size = lstm_hidden_size + char_size
        self.FeedForward = FeedforwardNet(ff_input_size, ff_hidden_size, ff_num_layers)

    def forward(self, macro_x, char_x, ret_x):
        """forward function

        Args:
            macro_x (tensor of shape[seq_len, input_size]): macroeconomic data from t1 to tn
            char_x (tensor of shape[input_size]): char data for a stock
        """
        macro_x = torch.unsqueeze(macro_x,1)
        hidden_g = self.MomentRNN(macro_x)
        hidden_g = torch.squeeze(hidden_g,0)
        hidden_g = hidden_g.repeat(char_x.shape[0],1)
        hidden_g_char = torch.cat((hidden_g, char_x), 1)
        out = self.FeedForward(hidden_g_char)
        out = torch.squeeze(out,1)
        return torch.mul(ret_x, out)

In [7]:
from Network.lstm import LSTM
from Network.feedforward_net import FeedforwardNet
class SDFNet(nn.Module):

    def __init__(self, macro_size, char_size, 
                 lstm_hidden_size, lstm_num_layers, 
                 ff_hidden_size, ff_num_layers):
        
        super().__init__()
        self.StateRNN = LSTM(macro_size, lstm_hidden_size, lstm_num_layers)
        ff_input_size = lstm_hidden_size + char_size
        self.FeedForward = FeedforwardNet(ff_input_size, ff_hidden_size, ff_num_layers)

    def forward(self, macro_x, char_x, ret_x):
        """forward function

        Args:
            macro_x (tensor of shape[seq_len, input_size]): macroeconomic data from t1 to tn
            char_x (tensor of shape[stock_num, input_size]): char data for a stock
            ret_x (tensor of shape[stock_num]): char data for a stock
        """

        macro_x = torch.unsqueeze(macro_x,1)
        hidden = self.StateRNN(macro_x)
        hidden = torch.squeeze(hidden,0)
        hidden = hidden.repeat(char_x.shape[0],1)
        hidden_char = torch.cat((hidden, char_x), 1)
        w = self.FeedForward(hidden_char)
        w = w/(torch.mean(w)*char_x.shape[0])
        w = torch.squeeze(w,1)
        sdf = 1-torch.dot(ret_x,w)
        return sdf,w
        
        
        

In [40]:
conditional_net = ConditionalNet(
    MACRO_SIZE, CHAR_SIZE, LSTM_HIDDEN, LSTM_LAYER, FF_HIDDEN, FF_LAYER)

In [29]:
sdf_net = SDFNet(
    MACRO_SIZE, CHAR_SIZE, LSTM_HIDDEN, LSTM_LAYER, FF_HIDDEN, FF_LAYER)

In [30]:
#macro_df = pd.read_csv("./data/124_macro_data.csv")
#feature_df = pd.read_csv("./data/27_features_rets_normalized.csv")

In [31]:
import numpy as np

In [32]:
macro_x = get_macro_by_date(macro_df, 197801, 201711)
feature_x, ret_x = get_feature_data_by_date(feature_df, 201711)
macro_x = torch.from_numpy(macro_x).float()
feature_x = torch.from_numpy(feature_x).float()
ret_x = torch.from_numpy(ret_x).float()


In [43]:
SDF,w = sdf_net(macro_x,feature_x,ret_x)

In [42]:
g_hat = conditional_net(macro_x,feature_x,ret_x)
g_hat.shape

torch.Size([3407])

In [51]:
loss = torch.sum((SDF*g_hat).pow(2))/g_hat.shape[0]

In [54]:
loss.backward()

In [26]:
torch.dot(torch.tensor([1,2,3]),torch.tensor([1,2,3]))

tensor(14)